# Water usage analysis code for trials

## Introduction

This notebook contains Python code that is able to run through multiple log files to determine shower usage. The log files should be in their own folder, one for each shower unit, and these folders should be place in the directory where this notebook is.

Main code that finds all the csv files in the directory folder and runs "water_usage_analysis" on them before producing the outputs. Check that the files are in the same location as glob.glob is looking for. Each /**/ represents a folder.

The wildcards allow for multiple folders to be analysed so long as the logs are the same depth in folders as eachother. E.g. 

./folder/shower1/log.csv … 

./folder/shower2/log.csv …

./folder/shower3/log.csv …

To run this notebook click Cell > Run All. Additionally ensure that the Kernel is cleared by clicking Kernel > Restart & Clear All or Restart & Run All 

The code will determine the water usage, shower on time (duration of a shower), and number of operations (button presses) per user. Shower on time and number of operations have been averaged hourly, water usage has been summed hourly. 

Once the code has completed running graphs in plotly will be produced as well as several CSV files. 

'operations_hourly.csv' is a csv file which contains the hourly average of the number of operations

'shower_on_time_hourly.csv' is a csv file which contains the hourly average of the shower on time
   

 'waterusage_hourly.csv' is a csv file which contains the total amount of water that was used in a given hour (hourly sum)
    

'full_usage_output.csv' is a csv file which contains the outputs of each individual shower including shower on time, number of operations, max motor temperature, date and time the shower ended, and the filename/location

'full_data_output.csv' is the full data output which has been processed and cleaned up. Removing extra columns. This is a csv file which has merged all the shower data into one and ordered based on date/time. Further analysis can be done using this csv file in Excel if desired. 

## Code walkthrough

Initialising steps and importing libraries

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Aug 22 12:01:29 2018

@author: Samvir Thandi for Kelda Technology

"""
import math                      # allows the use of math functions
import numpy as np               # allows use of the numpy library
import pandas as pd              # allows use of pandas data tools
import glob                      # allows use of Unix style file paths
import matplotlib.pyplot as plt
import plotly       # plotly library imports
import plotly.graph_objs as go

print(plotly.__version__)  # plotly version 

plotly.offline.init_notebook_mode(connected=True)  # plotly initialisation for Jupyter notebook use (offline mode)



This function converts the thermistor readings from the motor into degrees Celcius

In [ ]:
def thermistor_to_deg(x):
    """
    This function turns the theristor readings into a temperature reading in
    degrees Celsius
    """
    # conversion formula
    if x > 0 and x < 4:
        temp = (3610*298.0/(3610+298*math.log((x/(5-x)) * (33000/10000)))-273)
    else:
        temp = 0  # bug checking
    if temp > 150:  # ensures temp is not high > indicates corrupted data
        temp = 0
    else:
        pass
    return temp

This function processes the csv files and returns the dataframe and output series for analysis across all files

### Note
The date and timestamp for the line in a given csv file is only added after the new line key is pressed. Hence the date and time on a line actually refers to the date and time from the previous line. Due to this the columns are shifted down by two in this function. This is easier than shifting the datetime itself since it has become the index column when importing the csv file for ease of process later in the program. Shifting the index can cause issues and is more complex than simply shifting the rest of the columns. The reason the lines are shifted by two is to get out of the long term logging behaviour which can lead to large time deltas, so this ensures the logger is actually acting in short term mode.

In [ ]:
def water_usage_analysis(filename):
    """
    This function imports a csv file, parses the file for water usage
    """

    # Import the csv file skipping the first two rows because these do not
    # contain any data. Also set headers to none since the csv file does not
    # contain headers. Parse the dates from a string to datetime by combining
    # columns 0 and 1 together. Then picks the relevant columns.
    # skip lines which are considered 'bad' typically lines which are too long
    # This creates a pandas dataframe named 'df'

    df = pd.read_csv(filename,
                     skiprows=2,
                     header=None,
                     error_bad_lines=False,
                     parse_dates={'datetime': [0, 1]},
                     index_col='datetime'
                     )
    # rename the columns for ease of processing, including unused columns such that they can easily be identified later
    df.rename(columns={
                       2: '2',
                       3: '3',
                       4: '4',
                       5: '5',
                       6: '6',
                       7: '7',
                       8: '8',
                       9: '9',
                       10: '10',
                       11: 'RPS',
                       12: 'flowpulses',
                       13: '13',
                       14: '14',
                       15: 'tempprobe',
                       16: 'humidity',
                       17: '17',
                       18: '18',
                       19: '19',
                       20: '20',
                       21: 'temppcb',
                       22: '22',
                       23: '23',
                       24: '24',
                       25: '25',
                       26: 'motorthermistor',
                       27: '27',
                       28: '28',
                       29: '29',
                       30: '30',
                       31: '31',
                       32: 'fanspeed',
                       33: '33'
                       }, inplace=True)

    # drop extra columns as named by numbers which are actually strings so can be called easily. 
    df.drop(['2', '3', '4', '5', '6', '7', '8', '9', '10', '13', '14', '17',
             '18', '19', '20', '22', '23', '24', '25', '27', '28', '29', '30',
             '31', '33'], axis=1, inplace=True)
    # shift the revolution per second column down by one to compensate for the datetime being shifted
    df['RPS'] = df['RPS'].shift(2)
    # shift the flow pulses column down by two to compensate for the datetime being shifted
    df['flowpulses'] = df['flowpulses'].shift(2)
    # shift the temperature probe column down by two to compensate for the datetime being shifted
    df['tempprobe'] = df['tempprobe'].shift(2)
    # shift the humidity column down by one to compensate for the datetime being shifted
    df['humidity'] = df['humidity'].shift(2)
    # shift the pcb temperature column down by two to compensate for the datetime being shifted
    df['temppcb'] = df['temppcb'].shift(2)
    # shift the motor thermistor column down by two to compensate for the datetime being shifted
    df['motorthermistor'] = df['motorthermistor'].shift(2)
    # shift the fan speed setting column down by two to compensate for the datetime being shifted
    df['fanspeed'] = df['fanspeed'].shift(2)
    # drop lines with no data Not A Number or NaN, inplace to the current dataframe
    df.dropna(inplace=True)

    # processing on the flowpulses to convert to flow rate in l/min
    df['flowrate'] = 60*1000/df.flowpulses
    # Replace infinite flow rate with 0, this is due to the flow sensor giving an output of zero when there is no flow
    df.flowrate.replace(np.inf, 0, inplace=True)
    # determine the mean flow rate but excluding the zero values as this will skew the flow rate. 
    mean_flowrate = df[df.flowrate > 0].mean()['flowrate']

    # convert the thermistor readings into degrees
    df['motortemp'] = df['motorthermistor'].apply(thermistor_to_deg)
    # identify what the maximum motor temperature is
    max_motor_temp = df.motortemp.max()
    # Calculate the time delta between data points
    # first convert the index to date time, can cause errors otherwise
    # dt.seconds is used to convert the time delta into seconds for the
    # flow calculation
    # fillna(0) ensures first point delta is 0
    df['timedelta'] = pd.to_datetime(df.index)
    df['timedelta'] = df.timedelta.diff().dt.seconds.fillna(0)

    # Cumulative sum over the time deltas to work out how long the shower has
    # been running for. Useful for experiments where time is irrelevant
    df['duration'] = df.timedelta.cumsum()

    # calculates the amount of water used in the time delta (seconds) based on
    # the current flow rate (l/min)
    df['waterusage'] = df.flowrate * df.timedelta/60
    user_water_usage = df['waterusage'].sum()


    # determine the time shower was on and time spent in the shower
    shower_on_time = df[df.flowrate > 0].sum()['timedelta']/60
    # number of operations boolean logic.
    # If the current flow rate is greater than 0 and the previous flow rate is 0 then this is an operation
    operations = df[(df.flowrate > 0) &
                    (df.flowrate.shift(-1) == 0)].count()['flowrate']

    # saving the key datapoints as a output series, this is just a dataframe with only one line
    output_series = pd.Series({'datetime': df.index[-1],
                               'shower_on_time': shower_on_time,
                               'operations': operations,
                               'max_motor_temp': max_motor_temp,
                               'average_flow_rate': mean_flowrate,
                               'user_water_usage': user_water_usage,
                               'filename': filename
                               })
    # return the processed dataframe and the series
    return(df, output_series)

Main code that finds all the csv files in the directory folder and runs "water_usage_analysis" on them before producing the outputs. Check that the files are in the same location as glob.glob is looking for. Each /**/ represents a folder.

The wildcards allow for multiple folders to be analysed so long as the logs are the same depth in folders as eachother. E.g. 

./folder/shower1/log.csv … 

./folder/shower2/log.csv …

./folder/shower3/log.csv …



In [ ]:
# make a list of file names using the glob.glob module. This is where to define the depth of the files in the
# file structure 
filenames = glob.glob("./**/log*.csv")
# tells us number of users as the assumption can be made that a new user has entered after the csv file has been saved.
number_of_files = len(filenames)
# this is the list of series that will contain the output series from the previous function
outputs = []
# make a large dataframe which is all the local dataframes from each csv file combined into one.
dfs = []  # list of full dfs
for filename in filenames:
    # create a series with the outputs
    # error checking to ensure that the file has enough columns
    try:
        (df, output_series) = water_usage_analysis(filename)
    # excepts the keyerror and does not continue with this file
    except KeyError:
        print('File {} does not contain enough data columns'.format(filename))
        pass
    # value errors may appear, this checks for them and prints the filename
    except ValueError:
        print('Value error at {}'.format(filename))
        pass
    outputs.append(output_series)  # add the outputs to the list
    dfs.append(df)  # add the local df to the list of dfs

# shower time outputs
# turn the list of Pandas series called outputs into a Pandas dataframe
output_df = pd.DataFrame(outputs)
# parse the datetime column as actual datetime rather than just strings
output_df.datetime = pd.to_datetime(output_df['datetime'])
# set index to datetime
output_df.set_index('datetime', inplace=True)
# sort the index in date time order as processing occurs in a random order
output_df.sort_index(inplace=True)

# total data outputs
# concat the list of individual dataframes into one large dataframe
total_df = pd.concat(dfs)
# parse the datetime column as actual datetime rather than just strings
total_df.index = pd.to_datetime(total_df.index)
# sorts the dataframe by datetime as processing occurs in a random order
total_df.sort_index(inplace=True)

# totals
# the total water used is the sum of all the water usage from the combined dataframe
total_water = total_df.waterusage.sum()
# take an average of the shower on time column in the combined dataframe
tot_avg_dur_secs = output_df.shower_on_time.mean()*60
(average_dur_minutes, average_dur_seconds) = divmod(tot_avg_dur_secs, 60)
# motor data
maximum_motor_temp = output_df.max_motor_temp.max()
max_motor_temp_loc = output_df.loc[output_df.max_motor_temp ==
                                    output_df.max_motor_temp.max(),
                                    'filename'].values[0]
average_motor_temp = total_df.motortemp.mean()
fifty_perc_motor_temp = total_df.motortemp.quantile(0.5)
ninetyfifth_perc_motor_temp = total_df.motortemp.quantile(0.95)
ninetynineth_perc_motor_temp = total_df.motortemp.quantile(0.99)

# resample the df by hour and find the hourly average/sum
water_usage_hour = total_df.waterusage.resample('H').sum().fillna(0)
showerontime_hour = output_df.shower_on_time.resample('H').mean().fillna(0)
operations_hour = output_df.operations.resample('H').mean().fillna(0)

In [ ]:
    # save the outputs to csv files
    # the number of operations per hour
    operations_hour.to_csv('operations_hourly.csv')
    # the time the shower was on for
    showerontime_hour.to_csv('shower_on_time_hourly.csv')
    # how much water was used hourly
    water_usage_hour.to_csv('waterusage_hourly.csv')
    # shower on time, number of operations, motor temp for an individual user
    output_df.to_csv('individual_user_usage.csv')
    # all the data sorted by time, a combination of all csv files. This is a large file and may not be very useful for analysis
    total_df.to_csv('all_data_output.csv')

## Outputs

In [ ]:
with open('output.txt', 'w') as f:
    print("""\n\nFrom {} to {}\n\nNumber of users: {}
Total water consumption: {:.2f} litres

Average duration: {:.0f} minutes and {:.0f} seconds

The maximum motor temperature was {:.2f}°C in location {}
The average motor temperature is {:.2f}°C
The 50th percentile of the motor temperature is {:.2f}°C
The 95th percentile of the motor temperature is {:.2f}°C
The 99th percentile of the motor temperature is {:.2f}°C
    """.format(total_df.index[0], total_df.index[-1], number_of_files,
               total_water, average_dur_minutes, average_dur_seconds,
               maximum_motor_temp, max_motor_temp_loc,
               average_motor_temp, fifty_perc_motor_temp,
               ninetyfifth_perc_motor_temp, ninetynineth_perc_motor_temp), file=f)

In [ ]:
print("""\n\nFrom {} to {}\n\nNumber of users: {}
Total water consumption: {:.2f} litres

Average duration: {:.0f} minutes and {:.0f} seconds

The maximum motor temperature was {:.2f}°C in location {}
The average motor temperature is {:.2f}°C
The 50th percentile of the motor temperature is {:.2f}°C
The 95th percentile of the motor temperature is {:.2f}°C
The 99th percentile of the motor temperature is {:.2f}°C
    """.format(total_df.index[0], total_df.index[-1], number_of_files,
               total_water, average_dur_minutes, average_dur_seconds,
               maximum_motor_temp, max_motor_temp_loc,
               average_motor_temp, fifty_perc_motor_temp,
               ninetyfifth_perc_motor_temp, ninetynineth_perc_motor_temp))

## Plotly

Plotly is a graphing library which allows interactive plots to be generated. These plots can be saved to a plotly account and then exported to html or just an image if desired. Plots can be fully customised as desired using the key and value arguments in the dictionaries. For example the title key can be changed. Additionally under xaxis and yaxis is another dictionary where various parameters can be added such as font or size. A demonstatration for customising the axes can be found at https://plot.ly/python/axes/#set-and-style-axes-title-labels-and-ticks which should cover most customisation requirements. Additionally the full reference can be found at: https://plot.ly/python/reference/

In [ ]:
water_usage_hour['dates']=water_usage_hour.index

plotly.offline.plot({
    "data": [go.Scatter(
                        x=water_usage_hour.index,
                        y=water_usage_hour
                       )
            ],
    "layout": go.Layout(title="Hourly water consumption",
                        titlefont=dict(
                                        size=18
                                      ),
                        xaxis=dict(
                                    tickfont=dict(
                                                    size=14
                                                 )
                                  ),
                        yaxis=dict(
                                    title='Hourly Water consumption (litres)',
                                    titlefont=dict(
                                                    size=18
                                                  ),
                                    tickfont=dict(
                                                    size=14
                                                 )
                                   ),
                        annotations=[
                                     dict(
                                            showarrow=False,
                                            x=1,
                                            y=1.07,
                                            xref='paper',
                                            yref='paper',
                                            text='Total water consumption: {:.2f} litres'.format(total_water),
                                            font=dict(
                                                        size=14
                                                     )
                                         ),
                                     dict(
                                            showarrow=False,
                                            x=1,
                                            y=1,
                                            xref='paper',
                                            yref='paper',
                                            text='Number of users: {}'.format(number_of_files),
                                            font=dict(
                                                        size=14
                                                      )
                                         )
                                    ]
                       ),
}, filename='water_usage.html', auto_open=False)

In [ ]:
showerontime_hour['dates']=showerontime_hour.index

plotly.offline.plot({
    "data": [go.Scatter(
                        x=showerontime_hour.index,
                        y=showerontime_hour
                       )
            ],
    "layout": go.Layout(title="Average shower duration hourly",
                        titlefont=dict(
                                        size=18
                                      ),
                        xaxis=dict(
                                    tickfont=dict(
                                                    size=14
                                                 )
                                  ),
                        yaxis=dict(
                                    title='Shower duration (minutes)',
                                    titlefont=dict(
                                                    size=18 
                                                  ),
                                    tickfont=dict(
                                                    size=14
                                                 )
                                  )
                        )
}, filename='shower_time.html', auto_open=False)

In [ ]:
operations_hour['dates']=operations_hour.index

plotly.offline.plot({
    "data": [go.Scatter(
                        x=operations_hour.index,
                        y=operations_hour
                       )
            ],
    "layout": go.Layout(title="Average number of operations per user hourly",
                        titlefont=dict(
                                        size=18
                                      ),
                        xaxis=dict(
                                    tickfont=dict(
                                                 size=14
                                                )
                                  ),
                        yaxis=dict(
                                    title = 'Operations',
                                    titlefont=dict(
                                                   size=18
                                                  ),
                        tickfont=dict(
                                        size=14
                                     )
                                  )
                       )
}, filename='operations.html', auto_open=False)